In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data

import random, os, pathlib, time
from tqdm import tqdm
from sklearn import datasets

In [2]:
import nflib
from nflib.flows import SequentialFlow, NormalizingFlow, ActNorm, ActNorm2D, AffineConstantFlow
import nflib.coupling_flows as icf
import nflib.inn_flow as inn
import nflib.res_flow as irf

In [3]:
from torch import distributions
from torch.distributions import MultivariateNormal

In [4]:
# device = torch.device("cuda:0")
device = torch.device("cpu")

## MNIST dataset

In [5]:
import mylibrary.datasets as datasets
# import mylibrary.nnlib as tnn

In [6]:
mnist = datasets.FashionMNIST()
# mnist.download_mnist()
# mnist.save_mnist()
train_data, train_label_, test_data, test_label_ = mnist.load()

train_data = train_data / 255.
test_data = test_data / 255.

# train_label = tnn.Logits.index_to_logit(train_label_)
train_size = len(train_label_)

In [7]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)
test_label = torch.LongTensor(test_label_)

In [8]:
input_size = 784
output_size = 10

In [9]:
class MNIST_Dataset(data.Dataset):
    
    def __init__(self, data, label):
        self.data = data.reshape(-1, 1, 28, 28)
        self.label = label
        
#         self.label = mask.type(torch.float32).reshape(-1,1)
        self._shuffle_data_()
        
    def __len__(self):
        return len(self.data)
    
    def _shuffle_data_(self):
        randidx = random.sample(range(len(self.data)), k=len(self.data))
        self.data = self.data[randidx]
        self.label = self.label[randidx]
    
    def __getitem__(self, idx):
        img, lbl = self.data[idx], self.label[idx]
        return img, lbl

In [10]:
train_dataset = MNIST_Dataset(train_data, train_label)
test_dataset = MNIST_Dataset(test_data, test_label)

In [11]:
class ConnectedClassifier_Linear(nn.Module):
    
    def __init__(self,input_dim, num_sets, output_dim, inv_temp=1):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_sets = num_sets
        self.inv_temp = nn.Parameter(torch.ones(1)*inv_temp)
        
        self.linear = nn.Linear(input_dim, num_sets)
#         self.linear.bias.data *= 0
#         self.linear.weight.data *= 0.1
#         self.cls_weight = nn.Parameter(torch.randn(num_sets, output_dim)/output_dim)

        init_val = torch.randn(num_sets, output_dim)*0.01
        for ns in range(num_sets):
            init_val[ns, ns%output_dim] = 0.1
        self.cls_weight = nn.Parameter(init_val)
        
        self.cls_confidence = None
        
        
    def forward(self, x, hard=False):
#         self.cls_weight.data = torch.abs(self.cls_weight.data/self.cls_weight.data.sum(dim=1, keepdim=True))
        
        x = self.linear(x)
        if hard:
            x = torch.softmax(x*1e5, dim=1)
        else:
            x = torch.softmax(x*self.inv_temp, dim=1)
        self.cls_confidence = x
        c = torch.softmax(self.cls_weight, dim=1)
#         c = self.cls_weight
        return x@c ## since both are normalized, it is also normalized

In [12]:
class ConnectedClassifier_SoftKMeans(nn.Module):
    
    def __init__(self,input_dim, num_sets, output_dim, inv_temp=1):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_sets = num_sets
        self.inv_temp = nn.Parameter(torch.ones(1)*inv_temp)
        
        self.centers = nn.Parameter(torch.rand(num_sets, input_dim)*2-1)
        
#         self.cls_weight = nn.Parameter(torch.ones(num_sets, output_dim)/output_dim)

        init_val = torch.randn(num_sets, output_dim)*0.01
        for ns in range(num_sets):
            init_val[ns, ns%output_dim] = 0.1
        self.cls_weight = nn.Parameter(init_val)

        self.cls_confidence = None
        
        
    def forward(self, x, hard=False):
#         self.cls_weight.data = torch.abs(self.cls_weight.data/self.cls_weight.data.sum(dim=1, keepdim=True))
        
        x = x[:, :self.input_dim]
        dists = torch.cdist(x, self.centers)
        dists = dists/np.sqrt(self.input_dim) ### correction to make diagonal of unit square 1 in nD space
        
        if hard:
            x = torch.softmax(-dists*1e5, dim=1)
        else:
            x = torch.softmax(-dists*self.inv_temp, dim=1)
        self.cls_confidence = x
        c = torch.softmax(self.cls_weight, dim=1)
#         c = self.cls_weight
        return x@c ## since both are normalized, it is also normalized
#         return torch.softmax(x@self.cls_weight, dim=1)

In [13]:
# actf = irf.Swish
# flows = [
#     ActNorm(784),
#     irf.ResidualFlow(784, [784], activation=actf),
#     ActNorm(784),
#     irf.ResidualFlow(784, [784], activation=actf),
#     ActNorm(784),
#         ]

# model = SequentialFlow(flows)

In [14]:
actf = irf.Swish
flows = [
    ActNorm2D(1),
    irf.ConvResidualFlow(1, [16], activation=actf),
    irf.InvertiblePooling(2),
    ActNorm2D(4),
    irf.ConvResidualFlow(4, [64], activation=actf),
    irf.InvertiblePooling(2),
    ActNorm2D(16),
    irf.ConvResidualFlow(16, [64, 64], activation=actf),
    irf.Flatten(img_size=(16, 7, 7))
        ]

model = SequentialFlow(flows)

In [15]:
# model = nn.Sequential(nn.Linear(784, 784, bias=False),
#                       nn.BatchNorm1d(784),
#                       nn.SELU(),
#                       nn.Linear(784, 784, bias=False),
#                       nn.BatchNorm1d(784),
#                       nn.SELU(),
#                      )

In [16]:
# model(torch.randn(3, 1, 28, 28)).shape

In [17]:
model.to(device)

SequentialFlow(
  (flows): ModuleList(
    (0): ActNorm2D()
    (1): ConvResidualFlow(
      (resblock): ModuleList(
        (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): Swish()
        (2): Conv2d(16, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (2): InvertiblePooling()
    (3): ActNorm2D()
    (4): ConvResidualFlow(
      (resblock): ModuleList(
        (0): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): Swish()
        (2): Conv2d(64, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (5): InvertiblePooling()
    (6): ActNorm2D()
    (7): ConvResidualFlow(
      (resblock): ModuleList(
        (0): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): Swish()
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): Swish()
        (4): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )


In [18]:
classifier = ConnectedClassifier_SoftKMeans(784, 100, 10)
# classifier = ConnectedClassifier_Softmax(784, 100, 10)
classifier = classifier.to(device)

## Model Train

In [19]:
learning_rate = 0.0003
batch_size = 50

In [20]:
train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

In [21]:
# criterion = nn.NLLLoss()
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(list(model.parameters())+list(classifier.parameters()), lr=learning_rate)
# optimizer = optim.SGD(model.parameters(), lr=0.1)

print("number of params: ", sum(p.numel() for p in model.parameters()))

number of params:  60467


In [22]:
# for p in model.parameters():
#     print(torch.isnan(p).type(torch.float32).sum())

In [23]:
# model(torch.randn(10, 784).to(device)).shape

In [24]:
xx = iter(test_loader).next()[0]
xx.shape

torch.Size([50, 1, 28, 28])

In [25]:
losses = []
train_accs = []
test_accs = []
EPOCHS = 50

index = 0
for epoch in range(EPOCHS):
    train_acc = 0
    train_count = 0
    for xx, yy in tqdm(train_loader):
        xx, yy = xx.to(device), yy.to(device)
#     for xx, yy in tqdm(test_loader):

        yout = model(xx)
#         print(yout)
        yout = classifier(yout)    
#         print(yout)
        loss = criterion(yout, yy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(float(loss))

        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        train_acc += correct
        train_count += len(outputs)
#         break

    train_accs.append(float(train_acc)/train_count*100)
    train_acc = 0
    train_count = 0

    print(f'Epoch: {epoch}:{index},  Loss:{float(loss)}')
    test_count = 0
    test_acc = 0
    for xx, yy in tqdm(test_loader):
        xx, yy = xx.to(device), yy.to(device)
        with torch.no_grad():
            yout = classifier(model(xx))    
        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        test_acc += correct
        test_count += len(xx)
    test_accs.append(float(test_acc)/test_count*100)
    print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
    print()

### after each class index is finished training
print(f'\t-> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}')

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 0:0,  Loss:2.2961394786834717


  0%|          | 0/1200 [00:00<?, ?it/s]

Train Acc:22.61%, Test Acc:36.79%



  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

Train Acc:79.44%, Test Acc:79.40%



 60%|█████▉    | 714/1200 [00:54<00:37, 13.07it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/tsuman/Program_Files/Python/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-25-27b669bf3531>", line 10, in <module>
    for xx, yy in tqdm(train_loader):
  File "/home/tsuman/Program_Files/Python/miniconda3/lib/python3.7/site-packages/tqdm/std.py", line 1166, in __iter__
    for obj in iterable:
  File "/home/tsuman/Program_Files/Python/miniconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 435, in __next__
    data = self._next_data()
  File "/home/tsuman/Program_Files/Python/miniconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1068, in _next_data
    idx, data = self._get_data()
  File "/home/tsuman/Program_Files/Python/miniconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1034, in _get_data
    success, data = self._try_get_data()
  File "/hom

KeyboardInterrupt: 

In [ ]:
classifier.inv_temp

### Hard test accuracy with count per classifier

In [79]:
test_count = 0
test_acc = 0
set_count = torch.zeros(classifier.num_sets).to(device)
for xx, yy in tqdm(test_loader):
    xx, yy = xx.to(device), yy.to(device)
    with torch.no_grad():
        yout = classifier(model(xx), hard=True)
        set_indx, count = torch.unique(torch.argmax(classifier.cls_confidence, dim=1), return_counts=True) 
        set_count[set_indx] += count
    outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
    correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
    test_acc += correct
    test_count += len(xx)

print(f'Hard Test Acc:{float(test_acc)/test_count*100:.2f}%')
print(set_count.type(torch.long).tolist())

100%|██████████| 200/200 [00:01<00:00, 105.25it/s]

Hard Test Acc:88.00%
[0, 0, 0, 0, 1029, 0, 0, 1068, 1018, 0, 0, 0, 0, 0, 0, 1000, 0, 0, 0, 0, 0, 1010, 0, 0, 0, 0, 0, 0, 0, 1010, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 972, 0, 1011, 0, 0, 0, 0, 0, 0, 0, 0, 968, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 914, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### Hard train accuracy with count per classifier

In [80]:
test_count = 0
test_acc = 0
set_count = torch.zeros(classifier.num_sets).to(device)
for xx, yy in tqdm(train_loader):
    xx, yy = xx.to(device), yy.to(device)
    with torch.no_grad():
        yout = classifier(model(xx), hard=True)
        set_indx, count = torch.unique(torch.argmax(classifier.cls_confidence, dim=1), return_counts=True) 
        set_count[set_indx] += count
    outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
    correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
    test_acc += correct
    test_count += len(xx)

print(f'Hard Train Acc:{float(test_acc)/test_count*100:.2f}%')
print(set_count.type(torch.long).tolist())

100%|██████████| 1200/1200 [00:12<00:00, 99.51it/s] 

Hard Train Acc:92.36%
[0, 0, 0, 0, 6119, 0, 0, 6266, 6018, 0, 0, 0, 0, 0, 0, 6091, 0, 0, 0, 0, 0, 6070, 0, 0, 0, 0, 0, 0, 0, 5956, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5902, 0, 6053, 0, 0, 0, 0, 0, 0, 0, 0, 5927, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5598, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [81]:
#### Classifiers that enclose any data
torch.count_nonzero(set_count)

tensor(10)

In [58]:
#### classifier with class representation
torch.argmax(classifier.cls_weight, dim=1)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3,
        4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7,
        8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1,
        2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5,
        6, 7, 8, 9])

In [59]:
# The class labels are same as that of initialized
# tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3,
#         4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7,
#         8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1,
#         2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5,
#         6, 7, 8, 9], device='cuda:0')

In [60]:
classifier.cls_weight

Parameter containing:
tensor([[ 1.0948e+01, -2.6081e-01, -4.4869e-01, -1.1516e-01, -5.1962e-01,
         -5.4807e-01,  3.9923e-02, -5.8040e-01, -4.6139e-01, -6.0863e-01],
        [-3.3414e-01,  1.0927e+01, -4.2983e-01, -2.9968e-01, -4.2571e-01,
         -4.2072e-01, -4.7829e-01, -4.1128e-01, -3.7257e-01, -3.3730e-01],
        [-3.1180e-01, -5.1346e-01,  1.0906e+01, -4.3566e-01,  1.1364e-01,
         -4.7643e-01, -1.8677e-02, -5.7840e-01, -4.3652e-01, -5.3920e-01],
        [-2.1061e-01, -2.8361e-01, -5.5542e-01,  1.0987e+01, -3.4030e-01,
         -4.9548e-01, -3.5487e-01, -5.1462e-01, -4.1046e-01, -4.7260e-01],
        [-4.8059e-01, -5.7916e-01,  1.0865e-01, -2.7176e-01,  1.0955e+01,
         -6.9063e-01,  4.4545e-02, -6.3701e-01, -5.6656e-01, -6.5603e-01],
        [-4.4417e-01, -4.5547e-01, -4.1456e-01, -4.8265e-01, -5.1685e-01,
          1.0933e+01, -4.4972e-01, -1.1164e-01, -3.2782e-01, -1.9266e-01],
        [-5.5332e-02, -5.9996e-01,  3.3389e-02, -2.3910e-01,  1.4806e-01,
         -

In [133]:
# torch.unique(torch.argmax(classifier.cls_confidence, dim=1), return_counts=True)

In [134]:
classifier.inv_temp

Parameter containing:
tensor([12.8763], device='cuda:0', requires_grad=True)

In [135]:
### example output per classifier
yout[5]

tensor([1.4948e-08, 1.4584e-08, 1.4847e-08, 1.4654e-08, 1.4749e-08, 1.4790e-08,
        1.4930e-08, 1.4765e-08, 1.0000e+00, 1.4702e-08], device='cuda:0')

In [ ]:
asdfsdf ## to break the code

### analyze per classifier accuracy

In [136]:
test_count = 0
test_acc = 0
set_count = torch.zeros(classifier.num_sets).to(device)
set_acc = torch.zeros(classifier.num_sets).to(device)
for xx, yy in tqdm(train_loader):
    xx, yy = xx.to(device), yy.to(device)
    with torch.no_grad():
        yout = classifier(model(xx), hard=True)
        
    cls_indx = torch.argmax(classifier.cls_confidence, dim=1)
    set_indx, count = torch.unique(cls_indx, return_counts=True) 
    set_count[set_indx] += count
    
    outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
    correct = (outputs == yy.data.cpu().numpy()).astype(float)
    
    ### class_index has 100 possible values
    for i, c in enumerate(correct):
        set_acc[cls_indx[i]] += c
    
#     print(set_acc.sum(), set_count.sum())
#     break
    test_acc += correct.sum()
    test_count += len(xx)

print(f'Hard Train Acc:{float(test_acc)/test_count*100:.2f}%')
print(set_count.type(torch.long).tolist())

100%|██████████| 1200/1200 [00:03<00:00, 338.48it/s]

Hard Train Acc:94.20%
[1423, 8, 24, 431, 2139, 560, 200, 4761, 222, 964, 2139, 3743, 164, 776, 304, 336, 4, 0, 1404, 4, 189, 0, 791, 0, 23, 167, 1091, 0, 1267, 11, 1, 0, 64, 286, 1590, 66, 2641, 17, 6, 8, 1686, 617, 1095, 2522, 0, 67, 696, 52, 1089, 0, 138, 1283, 15, 129, 383, 2726, 714, 2, 413, 129, 233, 0, 34, 1555, 0, 1839, 22, 674, 98, 3696, 17, 6, 858, 367, 3, 28, 222, 37, 101, 39, 299, 290, 3252, 0, 65, 145, 50, 15, 574, 1167, 3, 0, 126, 137, 1043, 45, 103, 431, 830, 16]


In [137]:
set_acc/set_count

tensor([0.9452, 0.8750, 0.9583, 0.9211, 0.9158, 0.9982, 0.8650, 0.9853, 0.9775,
        0.9948, 0.8672, 0.9989, 0.8293, 0.8235, 0.9309, 0.9940, 0.7500,    nan,
        0.9957, 1.0000, 0.9524,    nan, 0.9039,    nan, 0.9565, 1.0000, 0.7562,
           nan, 0.9897, 0.9091, 1.0000,    nan, 0.7031, 0.9126, 0.9220, 0.9848,
        0.9023, 0.9412, 1.0000, 1.0000, 0.9419, 0.9984, 0.9461, 0.9623,    nan,
        1.0000, 0.9210, 1.0000, 0.9917,    nan, 0.9203, 0.9977, 0.8000, 0.8527,
        0.9399, 0.9938, 0.8151, 1.0000, 0.9831, 0.9922, 0.9185,    nan, 0.5000,
        0.9350,    nan, 0.9956, 0.9091, 0.9614, 1.0000, 0.9811, 0.9412, 1.0000,
        0.7401, 0.9619, 0.3333, 1.0000, 0.9414, 1.0000, 0.9307, 1.0000, 0.8060,
        0.9966, 0.8598,    nan, 0.9692, 1.0000, 0.7800, 0.9333, 1.0000, 0.9709,
        1.0000,    nan, 0.7302, 0.9562, 0.8897, 1.0000, 0.8155, 0.9536, 0.9892,
        1.0000], device='cuda:0')

In [138]:
for i, (cnt, acc, cls) in enumerate(zip(set_count.type(torch.long).tolist(),
                                   (set_acc/set_count).tolist(),
                                   torch.argmax(classifier.cls_weight, dim=1).tolist())):
    if cnt == 0: continue
    print(f"{i},\t {cnt},\t {cls}\t {acc*100:.2f}%")

0,	 1423,	 0	 94.52%
1,	 8,	 1	 87.50%
2,	 24,	 2	 95.83%
3,	 431,	 3	 92.11%
4,	 2139,	 4	 91.58%
5,	 560,	 5	 99.82%
6,	 200,	 6	 86.50%
7,	 4761,	 7	 98.53%
8,	 222,	 8	 97.75%
9,	 964,	 9	 99.48%
10,	 2139,	 0	 86.72%
11,	 3743,	 1	 99.89%
12,	 164,	 2	 82.93%
13,	 776,	 3	 82.35%
14,	 304,	 4	 93.09%
15,	 336,	 5	 99.40%
16,	 4,	 6	 75.00%
18,	 1404,	 8	 99.57%
19,	 4,	 9	 100.00%
20,	 189,	 0	 95.24%
22,	 791,	 2	 90.39%
24,	 23,	 4	 95.65%
25,	 167,	 5	 100.00%
26,	 1091,	 6	 75.62%
28,	 1267,	 8	 98.97%
29,	 11,	 9	 90.91%
30,	 1,	 0	 100.00%
32,	 64,	 2	 70.31%
33,	 286,	 3	 91.26%
34,	 1590,	 4	 92.20%
35,	 66,	 5	 98.48%
36,	 2641,	 6	 90.23%
37,	 17,	 7	 94.12%
38,	 6,	 8	 100.00%
39,	 8,	 9	 100.00%
40,	 1686,	 0	 94.19%
41,	 617,	 1	 99.84%
42,	 1095,	 2	 94.61%
43,	 2522,	 3	 96.23%
45,	 67,	 5	 100.00%
46,	 696,	 6	 92.10%
47,	 52,	 7	 100.00%
48,	 1089,	 8	 99.17%
50,	 138,	 0	 92.03%
51,	 1283,	 1	 99.77%
52,	 15,	 2	 80.00%
53,	 129,	 3	 85.27%
54,	 383,	 4	 93.99%
5